In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing

import data_describe as dd


path = '../data/'
data_name = ["train", 'test', 'sample_submission']

In [2]:
df = pd.read_csv("{}{}.csv".format(path,data_name[0]), encoding='utf8')
print(df.shape)

df_test = pd.read_csv("{}{}.csv".format(path,data_name[1]), encoding='utf8')
print(df_test.shape)

df_submission = pd.read_csv("{}{}.csv".format(path,data_name[2]), encoding='utf8')
print(df_submission.shape)

(1460, 81)
(1459, 80)
(1459, 2)


In [3]:
# 欄位空值與重複值描述
data_describe1 =dd.data_describe(df, 'numeric');
data_describe2 = dd.data_describe(df_test, 'numeric');

# test
# data_describe1.loc[['SalePrice', 'BsmtCond', 'MSSubClass'],'type']

---

# Tool

In [4]:

# 有序類別轉換為0-1數值

def order_cat_to_num(df, tar_list, map_list):
    '''
    from list to dict and make it's number between 0 to 1
    
    df: the dataframe we are going to deal with 
    tar_list: the list contain target columns 
    map_list: what u wanna change to numeric by order(from less to great)
    ''' 
    map_dict = dict((k,map_list.index(k)/(len(map_list)-1)) for k in map_list)
    for tar in tar_list:
        df[tar] = df[tar].map(map_dict) 

# test
# order_cat_to_num(df, ['ExterQual'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
# df['ExterQual']

In [5]:

# 空值為一類別

def NA_as_cat(df, tar_list, fill_with=-1):
    '''
    NULL number is kind of category in some column
    
    df: the dataframe we are going to deal with 
    tar_list: the list contain target columns 
    fill_with: what u wanna put inside null number
    ''' 
    for tar in tar_list:
        df.fillna({tar:fill_with}, inplace=True) 

# test
# NA_as_cat(df, ['BsmtQual', 'BsmtCond'], -1)
# df[['BsmtQual', 'BsmtCond']].head(20)

In [6]:

# 標準化數值欄位(n倍標準差)

def standardize_col(df, tar_list):
    '''
    standardize target columns
    
    df: the dataframe we are going to deal with 
    tar_list: the list contain target columns 
    '''
    for tar in tar_list:
        df[tar] = preprocessing.scale(df[tar].values, copy=False)

# test
# standardize_col(df, ['SalePrice', 'LotArea'])       
# df[['SalePrice', 'LotArea']].head(10)

In [7]:

# 類別資料攤平

def one_hot_encoding(df, tar_list, dummy_nan=True):
    '''
    do one-hot encoding for category types' data,
    
    df: the dataframe we are going to deal with 
    tar_list: the list contain target columns
    dummy_nan:  null will also take it as a category for default
    '''
    for tar in tar_list:
        df = pd.concat([df, pd.get_dummies(df[tar], prefix=tar, dummy_na=dummy_nan)], axis=1)
    df.drop(columns=tar_list, inplace=True)
    return df


# test
# df = one_hot_encoding(df, ['MSSubClass', 'MSZoning'])
# df.head(10)

In [8]:

# 類別資料圖檢視

def cat_plot(df, tar, compare_with = 'SalePrice'):
    '''
    use plot to find out the relation with y
    
    df: the dataframe we are going to deal with 
    tar: the column we want to compare with y
    compare_with: y
    '''
    # 類別分布數量
    sns.countplot(tar, data=df)
    plt.show()
    # 類別與價格分布(一)
    sns.catplot(data=df, x=tar, y=compare_with, kind='bar')
    plt.show()
    # 類別與價格分布(二)
    sns.boxplot(data=df, x=tar, y=compare_with)
    plt.show()

    
# test
# cat_plot(df, tar = 'SaleCondition', compare_with = 'SalePrice')

In [9]:

# 移除空值過多欄位

def drop_col(df_list, tar_list):
    '''
    drop the columns don't need
    
    df: the dataframe we are going to deal with 
    tar_list: the list contain target columns
    '''
    for df in df_list:
        df.drop(tar_list, inplace=True, axis=1)


# test        
# for df and df_test remove the columns 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', which have too much null numbers   

# drop_col([df,df_test], ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'])

---

# Table: Train

## 1. 有序類別

* Normal:   
        LotShape, Utilities, LandSlope, OverallQual, OverallCond, ExterQual, ExterCond, CentralAir, HeatingQC, KitchenQual     
* NA is a category:  
        BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageFinish, GarageQual, GarageCond,

* 有序類別數值化

In [10]:
# 有序類別轉數值

order_cat_to_num(df, ['BsmtQual','BsmtCond', 'GarageQual', 'GarageCond', 'ExterQual', 'ExterCond','HeatingQC', 
                      'KitchenQual','FireplaceQu']
                      , ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
order_cat_to_num(df, ['BsmtExposure'], ['No', 'Mn', 'Av', 'Gd'])
order_cat_to_num(df, ['LotShape'], ['IR3', 'IR2', 'IR1', 'Reg'])
order_cat_to_num(df, ['Utilities'], ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'])
order_cat_to_num(df, ['LandSlope'], ['Sev', 'Mod', 'Gtl'])
order_cat_to_num(df, ['OverallQual', 'OverallCond'], [1,2,3,4,5,6,7,8,9,10])
order_cat_to_num(df, ['GarageFinish'], ['Unf', 'RFn', 'Fin'])
order_cat_to_num(df, ['BsmtFinType1','BsmtFinType2'], ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'])
order_cat_to_num(df, ['CentralAir'], ['N', 'Y'])

In [11]:
# NA唯一類別，填入-1

tar_list = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
            'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond']

NA_as_cat(df, tar_list, -1)

## 2. 數值

* Normal:   
        SalePrice, LotArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal    
* 有空值:    
        LotFrontage, MasVnrArea, GarageYrBlt 

In [12]:
# 空值處理    應建模(KNN, EM)



# LotFrontage: 259 → 中位數填入    過多空值得欄位不應用中位數直接補
df.fillna({'LotFrontage':df['LotFrontage'].median()}, inplace=True)
# print(df[df['LotFrontage'].isnull()==True])    # test


# GarageYrBlt: 81 → 中位數填入
df['GarageYrBlt'] = np.where(df['GarageFinish'] == -1, -1, df['GarageYrBlt'])    # 沒有車庫的年份該如何處理
df['GarageCars'] = np.where(df['GarageFinish'] == -1, 0, df['GarageCars'])
df['GarageArea'] = np.where(df['GarageFinish'] == -1, 0, df['GarageArea'])
df['GarageQual'] = np.where(df['GarageFinish'] == -1, -1, df['GarageQual'])
df['GarageCond'] = np.where(df['GarageFinish'] == -1, -1, df['GarageCond'])


df.fillna({'GarageYrBlt':df['GarageYrBlt'].median()}, inplace=True)    # 剩餘空值中位數補
# print(df[df['GarageYrBlt'].isnull()==True])    # test

In [13]:
# if FireplaceQu == -1: Fireplaces = 0 
df['Fireplaces'] = np.where(df['FireplaceQu'] == -1, 0, df['Fireplaces'])

In [14]:
# 空值視為沒有

# if BsmtHalfBath == np.nan: BsmtHalfBath =0
df.fillna({'BsmtHalfBath':0}, inplace=True)

# if BsmtFullBath == np.nan: BsmtFullBath =0
df.fillna({'BsmtFullBath':0}, inplace=True)

In [15]:
# 空值代表typical，填入'Typ'
# if Functional == NULL: Functional = Typ
df.fillna({'Functional':'Typ'}, inplace=True)

In [16]:
# 沒有地下室，面積為零

# if BsmtFinType1 == -1: BsmtFinSF1 = 0  
df['BsmtFinSF1'] = np.where(df['BsmtFinType1'] == -1, 0, df['BsmtFinSF1'])

# if BsmtFinType2 == -1: BsmtFinSF12 = 0  
df['BsmtFinSF2'] = np.where(df['BsmtFinType2'] == -1, 0, df['BsmtFinSF2'])

df['TotalBsmtSF'] = np.where((df['BsmtFinType2'] == -1)&(df['BsmtFinType1'] == -1), 0, df['TotalBsmtSF'])
df['BsmtUnfSF'] = np.where((df['BsmtFinType2'] == -1)&(df['BsmtFinType1'] == -1), 0, df['BsmtUnfSF'])
df['BsmtFinSF1'] = np.where((df['BsmtFinType2'] == -1)&(df['BsmtFinType1'] == -1), 0, df['BsmtFinSF1'])


## 3. 類別

* one-hot-encoding:  
        MSSubClass, MSZoning, Street, LandContour, LotConfig, Neighborhood, Condition1, Condition2, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, Foundation, Heating, CentralAir(只有2類), PavedDrive, MoSold, SaleType, SaleCondition, YrSold, HouseStyle  
* 有空值:    
        MasVnrType, Electrical(FuseA, FuseF, FuseP有序; SBrkr, Mix無序。應拆分), Functional,   
* NA is a category: 
        GarageType   

In [17]:
#  'GarageType' 76筆空值為一類別
NA_as_cat(df, ['GarageType'], fill_with=-1)

In [18]:
# 空值處理

# MasVnrType: 8 → 眾數填入
df.fillna({'MasVnrType':df['MasVnrType'].mode()[0]}, inplace=True)
# print(df[df['MasVnrType'].isnull()==True])    # test

# MasVnrArea: 8 → if 'MasVnrType' is 'None', fill it with 0 , else fill it with mode
df['MasVnrArea'] = np.where(df['MasVnrType'] == 'None', 0, df['MasVnrArea'])
df.fillna({'MasVnrArea':df['MasVnrArea'].mode()[0]}, inplace=True)     
# print(df[df['MasVnrArea'].isnull()==True])    # test

In [19]:
# Electrical: 1 → 眾數填入
df.fillna({'Electrical':df['Electrical'].mode()[0]}, inplace=True)
# print(df[df['Electrical'].isnull()==True])    # test


# GarageType: 81 → 眾數填入
df.fillna({'GarageType':df['GarageType'].mode()[0]}, inplace=True)
# print(df[df['GarageType'].isnull()==True])    # test

## 4. 其他

* 刪除:  
        Alley, PoolQC, Fence, MiscFeature'  
 .  
* 年份跨度大: (作為數值)
        YearBuilt, YearRemodAdd, GarageYrBlt, 

In [20]:
# 類別數大於10的類別型資料

data_describe1[(data_describe1['duplicate']>10) & (data_describe1['type']=='category')]

,duplicate,is_null,null_number,null_rate,type
MSSubClass,15,False,0,0.0,category
Neighborhood,25,False,0,0.0,category
Exterior1st,15,False,0,0.0,category
Exterior2nd,16,False,0,0.0,category
MoSold,12,False,0,0.0,category


In [21]:
# 確認空值欄位
# check whether have null number left or not

data_describe1_2 =dd.data_describe(df, 'numeric');
data_describe1_2[data_describe1_2["is_null"]==True]

,duplicate,is_null,null_number,null_rate,type
Alley,3,True,1369,0.937671,numeric
PoolQC,4,True,1453,0.995205,numeric
Fence,5,True,1179,0.807534,numeric
MiscFeature,5,True,1406,0.963014,numeric


In [22]:
# 刪除空值過多欄位
df.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], inplace=True, axis=1)
df.shape    # check

(1460, 77)

## 5. 新增欄位

---

# Table: Test

## 1. 有序類別

* Normal:   
       LotShape, Utilities, LandSlope, OverallQual, OverallCond, ExterQual, ExterCond, CentralAir, HeatingQC     

* NA is a category:  
        BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageFinish, GarageQual, GarageCond, KitchenQual
          
* has null number:   
        Utilities,

In [23]:
# 有序類別轉數值

order_cat_to_num(df_test, ['BsmtQual','BsmtCond', 'GarageQual', 'GarageCond', 'ExterQual', 'ExterCond','HeatingQC', 
                      'KitchenQual', 'FireplaceQu']
                      , ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
order_cat_to_num(df_test, ['BsmtExposure'], ['No', 'Mn', 'Av', 'Gd'])
order_cat_to_num(df_test, ['LotShape'], ['IR3', 'IR2', 'IR1', 'Reg'])
order_cat_to_num(df_test, ['Utilities'], ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'])
order_cat_to_num(df_test, ['LandSlope'], ['Sev', 'Mod', 'Gtl'])
order_cat_to_num(df_test, ['OverallQual', 'OverallCond'], [1,2,3,4,5,6,7,8,9,10])
order_cat_to_num(df_test, ['GarageFinish'], ['Unf', 'RFn', 'Fin'])
order_cat_to_num(df_test, ['BsmtFinType1','BsmtFinType2'], ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'])
order_cat_to_num(df_test, ['CentralAir'], ['N', 'Y'])

In [24]:
# NA唯一類別，填入-1

tar_list = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
            'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'KitchenQual']

NA_as_cat(df_test, tar_list, fill_with=-1)

In [25]:
#   'Utilities' 僅兩筆空值，眾數填入
df_test.fillna({'Utilities':df['Utilities'].mode()[0]}, inplace=True)
# print(df_test[df_test['Utilities'].isnull()==True])    # test

## 2. 數值

* Normal:   
        SalePrice, LotArea, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal    
* 有空值:    
        LotFrontage, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, Fireplaces, GarageCars, GarageArea

In [26]:
# 空值處理


# LotFrontage: 259 → 中位數填入    過多空值得欄位不應用中位數直接補
df_test.fillna({'LotFrontage':df_test['LotFrontage'].median()}, inplace=True)
# print(df[df['LotFrontage'].isnull()==True])    # test


# GarageYrBlt: 81 → 中位數填入
df_test['GarageYrBlt'] = np.where(df_test['GarageFinish'] == -1, -1, df_test['GarageYrBlt'])    # 沒有車庫的年份該如何處理
df_test['GarageCars'] = np.where(df_test['GarageFinish'] == -1, 0, df_test['GarageCars'])
df_test['GarageArea'] = np.where(df_test['GarageFinish'] == -1, 0, df_test['GarageArea'])
df_test['GarageQual'] = np.where(df_test['GarageFinish'] == -1, -1, df_test['GarageQual'])
df_test['GarageCond'] = np.where(df_test['GarageFinish'] == -1, -1, df_test['GarageCond'])


df_test.fillna({'GarageYrBlt':df_test['GarageYrBlt'].median()}, inplace=True)    # 剩餘空值中位數補
# print(df[df['GarageYrBlt'].isnull()==True])    # test

In [27]:
# if FireplaceQu == -1: Fireplaces = 0 
df_test['Fireplaces'] = np.where(df_test['FireplaceQu'] == -1, 0, df_test['Fireplaces'])

In [28]:
# 空值視為沒有

# if BsmtHalfBath == np.nan: BsmtHalfBath =0
df_test.fillna({'BsmtHalfBath':0}, inplace=True)

# if BsmtFullBath == np.nan: BsmtFullBath =0
df_test.fillna({'BsmtFullBath':0}, inplace=True)

In [29]:
# 沒有地下室，面積為零

# 以中位數填空值
df_test.fillna({'BsmtFinSF2':df_test['BsmtFinSF2'].median()}, inplace=True)

# if BsmtFinType1 == -1: BsmtFinSF1 = 0  
df_test['BsmtFinSF1'] = np.where(df_test['BsmtFinType1'] == -1, 0, df_test['BsmtFinSF1'])

# if BsmtFinType2 == -1: BsmtFinSF12 = 0  
df['BsmtFinSF2'] = np.where(df['BsmtFinType2'] == -1, 0, df['BsmtFinSF2'])

df_test['TotalBsmtSF'] = np.where((df_test['BsmtFinType2'] == -1)&(df_test['BsmtFinType1'] == -1), 0, df_test['TotalBsmtSF'])
df_test['BsmtUnfSF'] = np.where((df_test['BsmtFinType2'] == -1)&(df_test['BsmtFinType1'] == -1), 0, df_test['BsmtUnfSF'])
df_test['BsmtFinSF1'] = np.where((df_test['BsmtFinType2'] == -1)&(df_test['BsmtFinType1'] == -1), 0, df_test['BsmtFinSF1'])

## 3. 類別

* one-hot-encoding:  
        MSSubClass,  Street, LandContour, LotConfig, Neighborhood, Condition1, Condition2, RoofStyle, RoofMatl,  Foundation, Heating, CentralAir(只有2類), PavedDrive, MoSold, SaleCondition, YrSold, Electrical(FuseA, FuseF, FuseP有序; SBrkr, Mix無序。應拆分), HouseStyle  
        
* 有空值:    
        MSZoning, MasVnrType, Functional, Exterior1st, Exterior2nd, SaleType  

* NA is a category: 
        GarageType   

In [30]:
#  'GarageType' 76筆空值為一類別

NA_as_cat(df_test, ['GarageType'], fill_with=-1)

# GarageType: 76 → 眾數填入
df_test.fillna({'GarageType':df_test['GarageType'].mode()[0]}, inplace=True)
# print(df[df['GarageType'].isnull()==True])    # test

In [31]:
# 空值處理

# SaleType: 1  → 歸類為其他
df_test.fillna({'SaleType':'Oth'}, inplace=True)

# MSZoning: 4  → 眾數填入
df_test.fillna({'MSZoning':df['MSZoning'].mode()[0]}, inplace=True)
# print(df_test[df_test['MSZoning'].isnull()==True])    # test


In [32]:
# 空值處理

# MasVnrType: 16  → 眾數填入
df_test.fillna({'MasVnrType':df_test['MasVnrType'].mode()[0]}, inplace=True)
# print(df_test[df_test['MasVnrType'].isnull()==True])    # test


# MasVnrArea: 8 → if 'MasVnrType' is 'None', fill it with 0 , else fill it with mode
df_test['MasVnrArea'] = np.where(df_test['MasVnrType'] == 'None', 0, df_test['MasVnrArea'])
df_test.fillna({'MasVnrArea':df_test['MasVnrArea'].mode()[0]}, inplace=True)     
# print(df_test[df_test['MasVnrArea'].isnull()==True])    # test

In [33]:
# Functional: 2  
df_test.fillna({'Functional':'Typ'}, inplace=True)
# print(df_test[df_test['Functional'].isnull()==True])    # test

In [34]:

# Exterior1st(mode)
df_test.fillna({'Exterior1st':df_test['Exterior1st'].mode()[0]}, inplace=True)

# Exterior2nd(mode)
df_test.fillna({'Exterior2nd':df_test['Exterior2nd'].mode()[0]}, inplace=True)

# Utilities(mode)
df_test.fillna({'Utilities':df_test['Utilities'].mode()[0]}, inplace=True)

# KitchenQual(mode)
df_test.fillna({'KitchenQual':df_test['KitchenQual'].mode()[0]}, inplace=True)

In [35]:
# MSZoning(多重複→移除)

## 4. 其他

* 刪除:  
        Alley, PoolQC, Fence, MiscFeature  
 .  
* 年份跨度大: 
        YearBuilt, YearRemodAdd, GarageYrBlt(has_null), 

In [36]:
# 類別數大於10的類別型資料

data_describe2[(data_describe2['duplicate']>10) & (data_describe2['type']=='category')]

,duplicate,is_null,null_number,null_rate,type
MSSubClass,16,False,0,0.000000,category
Neighborhood,25,False,0,0.000000,category
Exterior1st,14,True,1,0.000685,category
Exterior2nd,16,True,1,0.000685,category
MoSold,12,False,0,0.000000,category


In [37]:
# check whether have null number left or not

data_describe2_2 =dd.data_describe(df_test, 'numeric');
data_describe2_2[data_describe2_2["is_null"]==True]

,duplicate,is_null,null_number,null_rate,type
Alley,3,True,1352,0.926662,numeric
PoolQC,3,True,1456,0.997944,numeric
Fence,5,True,1169,0.801234,numeric
MiscFeature,4,True,1408,0.965045,numeric


In [38]:
# 刪除空職過多欄位
df_test.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], inplace=True, axis=1)
df_test.shape    # check

(1459, 76)

## 5. 新增欄位

---

# 轉存CSV _v2.1

In [39]:
# 未攤平與標準化
df.to_csv(path+data_name[0]+'_v2.1.csv', encoding='utf-8')
df_test.to_csv(path+data_name[1]+'_v2.1.csv', encoding='utf-8')

---

# one-hot-encoing

In [40]:
# tar_list = []

# one_hot_encoding(df_test, tar_list, True)

---

# 標準化

---

# 轉存CSV _v3.1

In [41]:
# df.to_csv(path+data_name[0]+'_v3.1.csv', encoding='utf-8')
# df_test.to_csv(path+data_name[1]+'_v3.1.csv', encoding='utf-8')